In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from darts import TimeSeries
from darts.utils.data import PastCovariatesSequentialDataset

from darts.dataprocessing.transformers import Scaler
from darts.models import BlockRNNModel

from darts.models import NBEATSModel

from darts.utils.likelihood_models import GaussianLikelihood
from darts.metrics import mape, rmse


### Data preprocessing

In [2]:
df = pd.read_csv('../data/retail_store_inventory.csv')
df['Date'] = pd.to_datetime(df['Date'])

df.drop(['Store ID', 'Category', 'Region', 'Inventory Level', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount', 'Competitor Pricing'], axis=1, inplace=True)

# Aggregate across all stores/ regions/ categories
aggregated_df = df.groupby(['Date', 'Product ID']).agg({ # Keep unique Date and Product ID pairs
    'Units Sold': 'sum',
    'Weather Condition': lambda x: x.mode()[0] if not x.mode().empty else None,  # Mode
    'Holiday/Promotion': 'max',  # If any promotion/holiday exists, it is recorded
    'Seasonality': lambda x: x.mode()[0] if not x.mode().empty else None  # Mode
}).reset_index()

# Aggregate on weekly baisis
aggregated_df['Week'] = aggregated_df['Date'].dt.to_period('W').astype(str)  # Create a 'Week' column based on the ISO calendar week

weekly_aggregated_df = aggregated_df.groupby(['Week', 'Product ID']).agg({
    'Units Sold': 'sum',
    'Weather Condition': lambda x: x.mode()[0] if not x.mode().empty else None,  # Mode
    'Holiday/Promotion': 'max',  # If any promotion/holiday exists, it is recorded
    'Seasonality': lambda x: x.mode()[0] if not x.mode().empty else None  # Mode
}).reset_index()

# Reformat the 'Week' column
weekly_aggregated_df['Week'] = pd.to_datetime(weekly_aggregated_df['Week'].str.split('/').str[0])  # Takes the first date of the week
weekly_aggregated_df = weekly_aggregated_df.sort_values(by=['Product ID', 'Week']) # Sort for consistency

# Encoding categorical variables as numerical for Darts
weekly_aggregated_df['Weather Condition'] = weekly_aggregated_df['Weather Condition'].astype('category').cat.codes
weekly_aggregated_df['Seasonality'] = weekly_aggregated_df['Seasonality'].astype('category').cat.codes

### Prepare Input for Darts

In [3]:
df = weekly_aggregated_df

# Train-test split based on forecast horizon
forecast_horizon = 20  # Forecast for the next 4 weeks
last_valid_time = df.set_index("Week")['Weather Condition'].last_valid_index() # Find the last available timestamp for future covariates
split_point = last_valid_time - pd.Timedelta(weeks=forecast_horizon)  # Compute the train-test split point (Leave `forecast_horizon` weeks for testing)
print(f"Split point: {split_point}")

target_series_dict = {}  # 'Units Sold'
past_covariates_dict = {}  # 'Weather Condition', 'Holiday/Promotion', 'Seasonality'

# Initialize lists for train/test sets
train_series_list = []
test_series_list = []
past_covariates_list = []
product_id_list = []

for product_id in df['Product ID'].unique():
    product_id_list.append(product_id)
    df_product = df[df['Product ID'] == product_id] # Select relevant rows for the product
    df_product = df_product.set_index('Week') # Set week as index
    
    # Create TimeSeries objects
    target_series = TimeSeries.from_dataframe(df_product, value_cols=['Units Sold'])
    past_covariates = TimeSeries.from_dataframe(df_product, 
                                                  value_cols=['Weather Condition', 'Holiday/Promotion', 'Seasonality'])
    
    train_series, test_series = target_series.split_after(split_point)

    # Convert to float32 for Darts compatibility
    train_series_list.append(train_series.astype(np.float32))
    test_series_list.append(test_series.astype(np.float32))
    past_covariates_list.append(past_covariates.astype(np.float32))

target_scaler = Scaler()
past_covariates_scaler = Scaler()

# Fit only on the training set, then transform train & test sets -- so the model does not see future information while scaling
train_series_scaled = [target_scaler.fit_transform(ts) for ts in train_series_list]
test_series_scaled = [target_scaler.transform(ts) for ts in test_series_list]  
full_series_scaled = [train.append(test) for train, test in zip(train_series_scaled, test_series_scaled)]

past_covariates_scaled = [past_covariates_scaler.fit_transform(ts) for ts in past_covariates_list]

Split point: 2023-08-14 00:00:00


### Customized Dataset Class

In [ ]:
class PastCovariatesSequentialDataset_aggregate_target(PastCovariatesSequentialDataset):
    def __getitem__(self, idx):
        past_target, past_covariates, static_covariates, future_covariates, future_target = super().__getitem__(idx)

        monthly_demand = np.sum(future_target, axis=0, keepdims=True)

        return past_target, past_covariates, static_covariates, future_covariates, monthly_demand
    
    
    def to_dataframe(self, limit=None):
        """
        Return all windowed samples as a flattened DataFrame.
        Each element in past_target, past_covariates, and future_target becomes its own column.
        """
        n = len(self)
        if limit is not None:
            n = min(n, limit)
        
        all_rows = []
        for i in range(n):
            past_target, past_covariates, _, _, future_target = self[i]
            
            row = {"sample": i}
            
            # Flatten and add past_target columns
            pt_vals = past_target.flatten().tolist()
            for j, val in enumerate(pt_vals):
                row[f"past_target_{j+1}"] = val

            # Flatten and add future_target columns
            ft_vals = future_target.flatten().tolist()
            for j, val in enumerate(ft_vals):
                row[f"future_target_{j+1}"] = val

            # Flatten and add past_covariates columns (assume univariate covariates)
            pc_vals = past_covariates.flatten().tolist()
            for j, val in enumerate(pc_vals):
                row[f"past_covariates_{j+1}"] = val
            
            all_rows.append(row)

        return pd.DataFrame(all_rows)

In [ ]:
input_chunk_length = 12
output_chunk_length = 4 

dataset = PastCovariatesSequentialDataset(
    target_series=train_series_scaled,
    covariates=past_covariates_scaled,
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length
)

dataset_aggregate = PastCovariatesSequentialDataset_aggregate_target(
    target_series=train_series_scaled,
    covariates=past_covariates_scaled,
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length
)

# === Inspect window ===
window_past_target, window_past_covariates, _, _, window_future_target = dataset[0]  # first window
window_past_target_aggregate, window_past_covariates_aggregate, _, _, window_future_target_aggregate = dataset_aggregate[0]  # first window

print("\n🔍 Inspecting future target:")
print("\nDefault window:")
print(window_future_target)

print("\nAggregate window:")
print(window_future_target_aggregate)
# === Inspect window ===


🔍 Inspecting future target:

Default window:
[[0.8541549 ]
 [0.64518565]
 [0.59694743]
 [0.541172  ]]

Aggregate window:
[[2.63746]]
[[0.33333334 0.         0.6666667 ]
 [0.         0.         0.        ]
 [0.33333334 0.         0.6666667 ]
 [0.         0.         0.6666667 ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.33333334]
 [0.33333334 0.         0.33333334]
 [0.         0.         0.33333334]
 [0.33333334 0.         0.        ]
 [0.33333334 0.         0.        ]
 [0.33333334 0.         0.        ]]


In [56]:
df_window = dataset_aggregate.to_dataframe()
df_window.head()

,sample,past_target_1,past_target_2,past_target_3,past_target_4,past_target_5,past_target_6,past_target_7,past_target_8,past_target_9,past_target_10,past_target_11,past_target_12,future_target_1,past_covariates_1,past_covariates_2,past_covariates_3,past_covariates_4,past_covariates_5,past_covariates_6,past_covariates_7,past_covariates_8,past_covariates_9,past_covariates_10,past_covariates_11,past_covariates_12,past_covariates_13,past_covariates_14,past_covariates_15,past_covariates_16,past_covariates_17,past_covariates_18,past_covariates_19,past_covariates_20,past_covariates_21,past_covariates_22,past_covariates_23,past_covariates_24,past_covariates_25,past_covariates_26,past_covariates_27,past_covariates_28,past_covariates_29,past_covariates_30,past_covariates_31,past_covariates_32,past_covariates_33,past_covariates_34,past_covariates_35,past_covariates_36
0,0,0.593933,0.573205,0.662333,0.570379,0.484831,0.468249,0.860373,0.540041,0.581873,0.410025,0.851705,0.769173,2.637460,0.333333,0.0,0.666667,0.000000,0.0,0.000000,0.333333,0.0,0.666667,0.000000,0.0,0.666667,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.333333,0.000000,0.0,0.333333,0.333333,0.0,0.000000,0.333333,0.0,0.000000,0.333333,0.0,0.000000
1,1,0.801583,0.593933,0.573205,0.662333,0.570379,0.484831,0.468249,0.860373,0.540041,0.581873,0.410025,0.851705,2.865461,0.000000,0.0,0.333333,0.333333,0.0,0.666667,0.000000,0.0,0.000000,0.333333,0.0,0.666667,0.000000,0.0,0.666667,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.333333,0.000000,0.0,0.333333,0.333333,0.0,0.000000,0.333333,0.0,0.000000
2,2,0.600716,0.801583,0.593933,0.573205,0.662333,0.570379,0.484831,0.468249,0.860373,0.540041,0.581873,0.410025,3.120219,0.333333,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.666667,0.000000,0.0,0.000000,0.333333,0.0,0.666667,0.000000,0.0,0.666667,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.333333,0.000000,0.0,0.333333,0.333333,0.0,0.000000
3,3,0.537969,0.600716,0.801583,0.593933,0.573205,0.662333,0.570379,0.484831,0.468249,0.860373,0.540041,0.581873,2.885058,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.666667,0.000000,0.0,0.000000,0.333333,0.0,0.666667,0.000000,0.0,0.666667,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.333333,0.000000,0.0,0.333333
4,4,0.556058,0.537969,0.600716,0.801583,0.593933,0.573205,0.662333,0.570379,0.484831,0.468249,0.860373,0.540041,2.612776,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.666667,0.000000,0.0,0.000000,0.333333,0.0,0.666667,0.000000,0.0,0.666667,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.333333,0.0,0.333333


### Instantiate & Fit model

In [9]:
add_encoders = {
    'cyclic': {'future': ['month']},   # this will add sin/cos month features
    'datetime_attribute': {'future': ['year', 'month', 'weekofyear']},  # optional, if you also want year, week etc.
}

In [ ]:
model_nbeats = NBEATSModel(
    input_chunk_length=12,
    output_chunk_length=4,
    add_encoders=add_encoders, 
    n_epochs=10, 
)
model_nbeats.fit_from_dataset(dataset, verbose=True)

# === Print forecast ===
forecast_horizon = 4
pred = model_nbeats.predict(n=forecast_horizon, 
                    series=train_series_scaled,
                    past_covariates=past_covariates_scaled)
print(pred[0])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 6.4 M  | train
-------------------------------------------------------------
6.4 M     Trainable params
1.6 K     Non-trainable params
6.4 M     Total params
25.545    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<TimeSeries (DataArray) (Week: 4, component: 1, sample: 1)> Size: 16B
array([[[0.6388899 ]],

       [[0.71163917]],

       [[0.72174066]],

       [[0.6380856 ]]], dtype=float32)
Coordinates:
  * Week       (Week) datetime64[ns] 32B 2023-08-21 2023-08-28 ... 2023-09-11
  * component  (component) object 8B 'Units Sold'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None


In [ ]:
model_nbeats_aggregate = NBEATSModel(
    input_chunk_length=12,
    output_chunk_length=1,   # Aggregate to 1 month
    add_encoders=add_encoders, 
    n_epochs=10, 
)
model_nbeats_aggregate.fit_from_dataset(dataset_aggregate, verbose=True)  # Fit on the aggregated dataset

# === Print forecast ===
forecast_horizon = 1  # Aggregate to 1 month
pred = model_nbeats_aggregate.predict(n=forecast_horizon, 
                    series=train_series_scaled,
                    past_covariates=past_covariates_scaled)
print(pred[0])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<TimeSeries (DataArray) (Week: 1, component: 1, sample: 1)> Size: 4B
array([[[2.5738595]]], dtype=float32)
Coordinates:
  * Week       (Week) datetime64[ns] 8B 2023-08-21
  * component  (component) object 8B 'Units Sold'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None


In [ ]:
model_rnn = BlockRNNModel(
    model="LSTM",              
    input_chunk_length=12,     
    output_chunk_length=4,
    n_epochs=10,             
    likelihood=GaussianLikelihood(),     
    add_encoders=add_encoders, 
)
model_rnn.fit_from_dataset(dataset, verbose=True)

# === Print forecast ===
forecast_horizon = 4
pred = model_rnn.predict(n=forecast_horizon, 
                    series=train_series_scaled,
                    past_covariates=past_covariates_scaled)
print(pred[0])

<TimeSeries (DataArray) (Week: 4, component: 1, sample: 1)> Size: 16B
array([[[0.65983105]],

       [[0.7807355 ]],

       [[0.2835676 ]],

       [[0.6193588 ]]], dtype=float32)
Coordinates:
  * Week       (Week) datetime64[ns] 32B 2023-08-21 2023-08-28 ... 2023-09-11
  * component  (component) object 8B 'Units Sold'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None


In [18]:
model_rnn_aggregate = BlockRNNModel(
    model="LSTM",              
    input_chunk_length=12,     
    output_chunk_length=1, # Aggregate to 1 month
    n_epochs=10,             
    likelihood=GaussianLikelihood(),     
    add_encoders=add_encoders, 
)
model_rnn_aggregate.fit_from_dataset(dataset_aggregate, verbose=True)

# === Print forecast ===
forecast_horizon = 1  # Aggregate to 1 month
pred = model_rnn_aggregate.predict(n=forecast_horizon, 
                    series=train_series_scaled,
                    past_covariates=past_covariates_scaled)
print(pred[0])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 3.1 K  | train
6 | fc              | Sequential       | 52     | train
-------------------------------------------------------------
3.2 K     Trainable params
0         Non-trainable params
3.2 K     Total params
0.013     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<TimeSeries (DataArray) (Week: 1, component: 1, sample: 1)> Size: 4B
array([[[2.7239957]]], dtype=float32)
Coordinates:
  * Week       (Week) datetime64[ns] 8B 2023-08-21
  * component  (component) object 8B 'Units Sold'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None


### Evaluate model

In [34]:
output_dir = 'models_test'

models = [model_nbeats, model_rnn]
model_names = ['NBEATS', 'RNN' ]

metrics = []

for model, model_name in zip(models, model_names):  # for each model
    model_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)

    for idx, full_ts in enumerate(full_series_scaled):  # for each product
        forecast = model.historical_forecasts(
            series=full_ts,
            past_covariates=past_covariates_scaled[idx],
            start=train_series_scaled[idx].end_time(),
            forecast_horizon= 4, 
            retrain=False,
            verbose=True
        )

        test_slice = full_ts.slice_intersect(forecast)
        truth_df = test_slice.pd_dataframe()
        forecast_df = forecast.pd_dataframe()
        forecast_df.to_csv(os.path.join(model_dir, f'forecast_product_{idx}.csv'))
        truth_df.to_csv(os.path.join(model_dir, f'ground_truth_product_{idx}.csv'))

        # Compute future 4 weeks
        truth_df['Future_4Week_Sum'] = (
            truth_df['Units Sold'][::-1]
            .rolling(window=4)
            .sum()[::-1]
        )
        truth = TimeSeries.from_dataframe(truth_df[:-3], value_cols='Future_4Week_Sum')
        forecast_df['Future_4Week_Sum'] = (
            forecast_df['Units Sold'][::-1]
            .rolling(window=4)
            .sum()[::-1]
        )
        forecast = TimeSeries.from_dataframe(forecast_df[:-3], value_cols='Future_4Week_Sum')
        
        mape_score = mape(truth, forecast)
        rmse_score = rmse(truth, forecast)

        print(f"{model_name} - Product {idx}: MAPE = {mape_score:.2f}%  RMSE = {rmse_score:.4f}")

        metrics.append({
                'product_id': idx,
                'mape': mape_score,
                'rmse': rmse_score
                })

    pd.DataFrame(metrics).to_csv(os.path.join(model_dir, 'metrics_summary.csv'), index=False)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


NBEATS - Product 0: MAPE = 8.44%  RMSE = 0.2745


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


RNN - Product 18: MAPE = 12.80%  RMSE = 0.4643


Predicting: |          | 0/? [00:00<?, ?it/s]

RNN - Product 19: MAPE = 9.60%  RMSE = 0.3233


In [35]:
print(truth_df)
print(truth)

component   Units Sold  Future_4Week_Sum
Week                                    
2023-09-04    0.464052          2.602033
2023-09-11    0.723675          2.801198
2023-09-18    0.651416          2.621641
2023-09-25    0.762890          2.704067
2023-10-02    0.663217          2.518156
2023-10-09    0.544118          2.421750
2023-10-16    0.733842          2.654140
2023-10-23    0.576979          2.796660
2023-10-30    0.566812          2.941540
2023-11-06    0.776507          2.901416
2023-11-13    0.876362          2.754721
2023-11-20    0.721859          2.754721
2023-11-27    0.526688          2.681373
2023-12-04    0.629811          2.747821
2023-12-11    0.876362          2.035403
2023-12-18    0.648511               NaN
2023-12-25    0.593137               NaN
2024-01-01   -0.082607               NaN
<TimeSeries (DataArray) (Week: 15, component: 1, sample: 1)> Size: 120B
array([[[2.60203344]],

       [[2.8011983 ]],

       [[2.62164134]],

       [[2.70406693]],

       [[2.5

In [29]:
output_dir = 'models_test'

models = [model_nbeats_aggregate, model_rnn_aggregate]
model_names = ['NBEATS Aggregate', 'RNN Aggregate']

metrics = []

for model, model_name in zip(models, model_names):  # for each model
    model_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)

    for idx, full_ts in enumerate(full_series_scaled):  # for each product
        forecast = model.historical_forecasts(
            series=full_ts,
            past_covariates=past_covariates_scaled[idx],
            start=train_series_scaled[idx].end_time(),
            forecast_horizon= 1, # Aggregate to 1 month
            retrain=False,
            verbose=True
        )

        test_slice = full_ts.slice_intersect(forecast)
        test_df = test_slice.pd_dataframe()
        test_df['Future_4Week_Sum'] = (
            test_df['Units Sold'][::-1]
            .rolling(window=4)
            .sum()[::-1]
        )
        # Remove the last 3 weeks of data (NA)
        test_df = test_df[:-3]
        truth = TimeSeries.from_dataframe(test_df, value_cols='Future_4Week_Sum')

        test_df.to_csv(os.path.join(model_dir, f'ground_truth_product_{idx}.csv'))
        forecast.pd_dataframe()[:-3].to_csv(os.path.join(model_dir, f'forecast_product_{idx}.csv'))

        mape_score = mape(truth, forecast)
        rmse_score = rmse(truth, forecast)
        print(f"{model_name} - Product {idx}: MAPE = {mape_score:.2f}%  RMSE = {rmse_score:.4f}")

        metrics.append({
                    'product_id': idx,
                    'mape': mape_score,
                    'rmse': rmse_score
                })

    pd.DataFrame(metrics).to_csv(os.path.join(model_dir, 'metrics_summary.csv'), index=False)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


NBEATS Aggregate - Product 0: MAPE = 8.95%  RMSE = 0.2831


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

RNN Aggregate - Product 18: MAPE = 17.40%  RMSE = 0.5207


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

RNN Aggregate - Product 19: MAPE = 14.56%  RMSE = 0.4264


In [27]:
test_df = test_slice.pd_dataframe()
test_df['Future_4Week_Sum'] = (
    test_df['Units Sold'][::-1]
    .rolling(window=4)
    .sum()[::-1]
)
test_df.head


<bound method NDFrame.head of component   Units Sold  Future_4Week_Sum
Week                                    
2023-08-14    0.548112          2.088235
2023-08-21    0.518519          2.263798
2023-08-28    0.557553          2.396696
2023-09-04    0.464052          2.602033
2023-09-11    0.723675          2.801198
2023-09-18    0.651416          2.621641
2023-09-25    0.762890          2.704067
2023-10-02    0.663217          2.518156
2023-10-09    0.544118          2.421750
2023-10-16    0.733842          2.654140
2023-10-23    0.576979          2.796660
2023-10-30    0.566812          2.941540
2023-11-06    0.776507          2.901416
2023-11-13    0.876362          2.754721
2023-11-20    0.721859          2.754721
2023-11-27    0.526688          2.681373
2023-12-04    0.629811          2.747821
2023-12-11    0.876362          2.035403
2023-12-18    0.648511               NaN
2023-12-25    0.593137               NaN
2024-01-01   -0.082607               NaN>